In [9]:
"""
Convert commonvoice into wav and create the training and test json files
"""
import os
import json
import random
import csv
from pydub import AudioSegment

data = []
args = {
    "file_path": "cv-corpus-sample/en/validated.tsv",
    "percent": 20,
    "convert": False,
    "save_json_path": "data/",
    
}

directory = args["file_path"].rpartition('/')[0]
percent = args["percent"]

with open(args["file_path"]) as f:
    length = sum(1 for line in f)

with open(args["file_path"], newline='') as csvfile: 
    reader = csv.DictReader(csvfile, delimiter='\t')
    index = 1
    if(args["convert"]):
        print(str(length) + "files found")
    for row in reader:  
        file_name = row['path']
        filename = file_name.rpartition('.')[0] + ".wav"
        text = row['sentence']
        if(args["convert"]):
            data.append({
            "key": directory + "/clips/" + filename,
            "text": text
            })
            print("converting file " + str(index) + "/" + str(length) + " to wav", end="\r")
            src = directory + "/clips/" + file_name
            dst = directory + "/clips/" + filename
            sound = AudioSegment.from_mp3(src)
            sound.export(dst, format="wav")
            index = index + 1
        else:
            data.append({
            "key": directory + "/clips/" + file_name,
            "text": text
            })
random.shuffle(data)

f = open(args["save_json_path"] +"/"+ "train.json", "w")

with open(args["save_json_path"] +"/"+ 'train.json','w') as f:
    d = len(data)
    i=0
    while(i<int(d - d/percent)):
        r=data[i]
        line = json.dumps(r)
        f.write(line + "\n")
        i = i+1
        
f = open(args["save_json_path"] +"/"+ "test.json", "w")
with open(args["save_json_path"] +"/"+ 'test.json','w') as f:
    d = len(data)
    i=int(d-d/percent)
    while(i<d):
        r=data[i]
        line = json.dumps(r)
        f.write(line + "\n")
        i = i+1